In [4]:
from datetime import datetime

In [1]:
# get ensemble
ensemble = c3.SimulationEnsemble.get({"id": 1})
# get path to ensemble in the file system
ensemble_path = c3.FileSystem.inst().rootUrl() + 'gordon-group/' + ensemble.name + '/'
# prepath to all files
prepath_to_all_files = ensemble_path + ensemble.prePathToFiles

In [2]:
# find simulation samples in that ensamble
samples = c3.SimulationSample.fetch({
    # filter criteria
    "filter": "ensemble == ensemble"
})
print('Found ', samples.count, ' simulation samples\n')

# isolate Sample Zero
sample_zero = samples.objs[0]
print('Sample zero:\n',sample_zero)

Found  221  simulation samples

Sample zero:
 c3.SimulationSample(
 id='acure_aircraft_No_0',
 meta=c3.Meta(
        tenantTagId=151,
        tenant='dev',
        tag='tc02d',
        created=datetime.datetime(2021, 9, 29, 21, 33, 43, tzinfo=datetime.timezone.utc),
        createdBy='provisioner',
        updated=datetime.datetime(2021, 9, 29, 21, 33, 43, tzinfo=datetime.timezone.utc),
        updatedBy='provisioner',
        timestamp=datetime.datetime(2021, 9, 29, 21, 33, 43, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='SimulationSample'),
 version=1,
 simulationNumber=0,
 ensemble=c3.SimulationEnsemble(id='1'),
 parameters=c3.SimulationModelParameters(id='acure_aircraft_No_0'))


In [3]:
path_to_sample_zero_files = prepath_to_all_files + str(sample_zero.simulationNumber).zfill(3) + '/'

# look at files for this simulation
file_list = c3.FileSystem.inst().listFiles(path_to_sample_zero_files).files
print('Sample', sample_zero.simulationNumber, 'contains', len(file_list), 'NetCDF files')

Sample 0 contains 53 NetCDF files


### One single file

In [17]:
# list returns AzureFile... we want File
file_obj = c3.File.makeObj({
    "url": file_list[0].url
})

# add it to table
SimOutFileObj = c3.SimulationOutputFile.create(this={
    "simulationSample": sample_zero,
    "file": file_obj
})

In [18]:
# clean it
c3.SimulationOutputFile.removeAll()

1

### All files in SimulationSample 0

In [19]:
for file in file_list:
    # create file object
    file_obj = c3.File.makeObj({
        "url": file.url
    })
    
    # upsert it
    SimOutFileObj = c3.SimulationOutputFile.upsert(this={
    "simulationSample": sample_zero,
    "file": file_obj
    })

In [27]:
# clean it
c3.SimulationOutputFile.removeAll()

1933

### All files in the entire SimulationEnsemble

In [5]:
for sample in samples.objs[0:1]:
    # get path to files
    path_to_sample_files = prepath_to_all_files + str(sample.simulationNumber).zfill(3) + '/'
    file_list = c3.FileSystem.inst().listFiles(path_to_sample_files).files
    print('Sample', sample.simulationNumber, 'contains', len(file_list), 'NetCDF files')
    # upsert them
    for file in file_list:
        if file.url[-3:] == '.nc':
            # create file object
            file_obj = c3.File.makeObj({
                "url": file.url
            })
        
            # get date
            date_str = file.url[-11:-3]
            date_dt = datetime.strptime(date_str, '%Y%m%d')
    
            # upsert it
            SimOutFileObj = c3.SimulationOutputFile.upsert(this={
                "simulationSample": sample,
                "file": file_obj,
                "dateTag": date_dt
            })

Sample 0 contains 53 NetCDF files


In [46]:
# clean all
c3.SimulationOutputFile.removeAll()

53

### Batch upserting

In [14]:
SimOutFileObjs = []
for sample in samples.objs:
    # get path to files
    path_to_sample_files = prepath_to_all_files + str(sample.simulationNumber).zfill(3) + '/'
    file_list = c3.FileSystem.inst().listFiles(path_to_sample_files).files
    print('Sample', sample.simulationNumber, 'contains', len(file_list), 'NetCDF files')
    # upsert them
    for file in file_list:
        if file.url[-3:] == '.nc':
            # create file object
            file_obj = c3.File.makeObj({
                "url": file.url
            })
        
            # get date
            date_str = file.url[-11:-3]
            date_dt = datetime.strptime(date_str, '%Y%m%d')
    
            # upsert it
            SimOutFileObjs.append(c3.SimulationOutputFile(**{
                "simulationSample": sample,
                "file": file_obj,
                "dateTag": date_dt
            }))

Sample 0 contains 53 NetCDF files
Sample 1 contains 53 NetCDF files
Sample 10 contains 53 NetCDF files
Sample 100 contains 54 NetCDF files
Sample 101 contains 54 NetCDF files
Sample 102 contains 54 NetCDF files
Sample 103 contains 54 NetCDF files
Sample 104 contains 54 NetCDF files
Sample 105 contains 54 NetCDF files
Sample 106 contains 54 NetCDF files
Sample 107 contains 54 NetCDF files
Sample 108 contains 53 NetCDF files
Sample 109 contains 53 NetCDF files
Sample 11 contains 53 NetCDF files
Sample 110 contains 53 NetCDF files
Sample 111 contains 53 NetCDF files
Sample 112 contains 53 NetCDF files
Sample 113 contains 53 NetCDF files
Sample 114 contains 53 NetCDF files
Sample 115 contains 53 NetCDF files
Sample 116 contains 53 NetCDF files
Sample 117 contains 53 NetCDF files
Sample 118 contains 53 NetCDF files
Sample 119 contains 53 NetCDF files
Sample 12 contains 53 NetCDF files
Sample 120 contains 54 NetCDF files
Sample 121 contains 54 NetCDF files
Sample 122 contains 54 NetCDF files

In [15]:
c3.SimulationOutputFile.upsertBatch(SimOutFileObjs)

c3.ObjList<SimulationOutputFile>(
 stats=c3.Stats(
         createdObjCount=11718,
         updatedObjCount=0,
         removedObjCount=0,
         failedObjCount=0,
         objCount=11718,
         dbWait=0.0,
         time=59.219,
         dbTime=55.835527464,
         objsPerSecond=197.87568179131696,
         dbObjsPerSecond=209.86637956550493),
 objs=c3.Arry<SimulationOutputFile>([c3.SimulationOutputFile(
         id='fb4a9313-3261-4b30-8b01-3208952f2165',
         meta=c3.Meta(
                created=datetime.datetime(2021, 9, 30, 18, 1, 21, tzinfo=datetime.timezone.utc),
                updated=datetime.datetime(2021, 9, 30, 18, 1, 21, tzinfo=datetime.timezone.utc),
                timestamp=datetime.datetime(2021, 9, 30, 18, 1, 21, tzinfo=datetime.timezone.utc)),
         version=1),
        c3.SimulationOutputFile(
         id='2461e526-f876-4467-8d36-847e30033323',
         meta=c3.Meta(
                created=datetime.datetime(2021, 9, 30, 18, 1, 21, tzinfo=datetime.timez

In [13]:
# clean it
c3.SimulationOutputFile.removeAll()

53